In [1]:
# !pip install tabulate
# !pip install psutil
# !pip install elasticsearch

In [1]:
import platform
import psutil
import os
from datetime import datetime
from tabulate import tabulate

import elasticsearch
from elasticsearch import Elasticsearch

import json

import time

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
es.ping()

/home/sherlock/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


True

In [4]:
# database_name = 'pid_index'
# database_name = 'pid_index_new'
database_name = 'new'
cpu_greater = 'cpu_greater'
mem_greater = 'mem_greater'
count_db = 'count_db'

In [7]:
es.indices.create(index=database_name, ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'new'}

In [8]:
es.indices.create(index=cpu_greater, ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cpu_greater'}

In [9]:
es.indices.create(index=mem_greater, ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mem_greater'}

In [10]:
es.indices.create(index=count_db, ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'count_db'}

In [11]:
es.info()

{'name': 'arpit',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'lRupjgnrQGCK1Ataaqeuyg',
 'version': {'number': '7.16.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb',
  'build_date': '2021-12-18T19:42:46.604893745Z',
  'build_snapshot': False,
  'lucene_version': '8.10.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [12]:
print('General info about processes')
psutil.test()

General info about processes
USER         PID  %MEM     VSZ     RSS  NICE STATUS  START   TIME  CMDLINE
root           1   0.1  166.0M    8.0M        sleep  13:47  52:52  /sbin/init sp
root           2   0.0    0.0B    0.0B        sleep  13:47  30:00  kthreadd
root           3   0.0    0.0B    0.0B   -20   idle  13:47  30:00  rcu_gp
root           4   0.0    0.0B    0.0B   -20   idle  13:47  30:00  rcu_par_gp
root           6   0.0    0.0B    0.0B   -20   idle  13:47  30:00  kworker/0:0H-
root           9   0.0    0.0B    0.0B   -20   idle  13:47  30:00  mm_percpu_wq
root          10   0.0    0.0B    0.0B        sleep  13:47  30:00  rcu_tasks_rud
root          11   0.0    0.0B    0.0B        sleep  13:47  30:00  rcu_tasks_tra
root          12   0.0    0.0B    0.0B        sleep  13:47  30:09  ksoftirqd/0
root          13   0.0    0.0B    0.0B         idle  13:47  30:28  rcu_sched
root          14   0.0    0.0B    0.0B        sleep  13:47  30:00  migration/0
root          15   0.0    0.0

In [16]:
sys_mem = psutil.virtual_memory().total #+ psutil.swap_memory().total
# used_mem = psutil.virtual_memory().used
print("Total Memory of this system is", round(sys_mem/(1024**3), 3), 'GB')
# print("Total Used Memory of this system right now is", round(used_mem/(1024**3), 3), 'GB')
# print("Percentage of memory used of this system right now is", round(used_mem/sys_mem*100, 3), '%')
print("Total CPU Usage of this system right now is", psutil.cpu_percent(), '%')

Total Memory of this system is 11.615 GB
Total CPU Usage of this system right now is 11.8 %


In [17]:
def add_to_elasticSearch(timestamp, database_name):
    #l_small = []
    mem_count = 0
    cpu_count = 0
    mem_threshold = 5.0
    cpu_threshold = 5.0
    
    for process in psutil.process_iter():
        pid = process.pid                     # getting pid
        cpu = process.cpu_percent()           # getting cpu percentage

        try:
            mem = process.memory_full_info()  # getting whole memory info
            mem_rss = mem.rss                 # unique set size(private memory)
            
        except psutil.AccessDenied:           # this access is denied for root files
            mem = 0
            mem_rss = 0        

        total_mem_p = round(mem_rss / sys_mem * 100, 3)  # calculating total memory percange for each process
        
        data = {'pid': pid,'CPU%': cpu, 'TotalMemory%': total_mem_p, 'TimeStamp': timestamp}

        # adding process information to ElasticSearch Database
        id_val = pid + timestamp
        res_u = es.index(index=database_name, document=data, id=id_val)
      
        if cpu > cpu_threshold:
            cpu_count += 1
            data_cpu = {'pid': pid, 'cpu_g': cpu, 'mem': total_mem_p, 'tstamp': timestamp}
            es.index(index=cpu_greater, document=data_cpu, id=timestamp)
            
        if total_mem_p > mem_threshold:
            mem_count += 1
            data_mem = {'pid': pid, 'cpu': cpu, 'mem_g': total_mem_p, 'tstamp': timestamp}
            es.index(index=mem_greater, document=data_mem, id=timestamp)
    
    data_c = {'countCPU': cpu_count, 'countMEM': mem_count, 'TimeIteration': timestamp}
    es.index(index=count_db, document=data_c, id=timestamp)
    
    print('All Saved to Elastic database for timestamp', timestamp)    

In [18]:
for i in range(50):
    add_to_elasticSearch(time.time(), database_name)
    
    waiting = 2
    print(f'waiting for {waiting} seconds, {i} iteration\n')
    time.sleep(waiting) # making program sleep fro 2 seconds 

All Saved to Elastic database for timestamp 1641224152.337997
waiting for 2 seconds, 0 iteration

All Saved to Elastic database for timestamp 1641224176.2684572
waiting for 2 seconds, 1 iteration

All Saved to Elastic database for timestamp 1641224221.3084671
waiting for 2 seconds, 2 iteration

All Saved to Elastic database for timestamp 1641224271.3722715
waiting for 2 seconds, 3 iteration

All Saved to Elastic database for timestamp 1641224311.1066313
waiting for 2 seconds, 4 iteration

All Saved to Elastic database for timestamp 1641224360.1564717
waiting for 2 seconds, 5 iteration

All Saved to Elastic database for timestamp 1641224395.4411752
waiting for 2 seconds, 6 iteration

All Saved to Elastic database for timestamp 1641224421.8164656
waiting for 2 seconds, 7 iteration

All Saved to Elastic database for timestamp 1641224447.9365647
waiting for 2 seconds, 8 iteration

All Saved to Elastic database for timestamp 1641224476.480481
waiting for 2 seconds, 9 iteration

All Saved to